In [1]:
import torch
from src.models import HTGPModel
from src.utils import HTGP_Calculator, HTGPConfig

model_path = r"Checkpoints\model_epoch_15.pt"
print(f"Loading checkpoint from: {model_path}")
device = "cuda"
checkpoint = torch.load(model_path, map_location=device, weights_only=False)

saved_config = None
if isinstance(checkpoint, dict) and 'model_config' in checkpoint:
    saved_config = checkpoint['model_config']
    print(saved_config)

if saved_config is not None:
    print(f"✅ Found configuration in checkpoint! Using saved config.")
    config = saved_config
else:
    print(f"⚠️ No config found in checkpoint. Falling back to hardcoded default config.")
    config = HTGPConfig(
        num_atom_types=55, 
        hidden_dim=128, 
        num_layers=2, 
        cutoff=7.0, 
        num_rbf=10,
        use_L0=True, 
        use_L1=True,
        use_L2=True, 
        use_gating=True, 
        use_long_range=False
    )

d:\software\miniconda\envs\env_test_1\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: D:\software\miniconda\envs\env_test_1\Lib\site-packages\torch_scatter\_version_cuda.pyd
  import torch_geometric.typing
d:\software\miniconda\envs\env_test_1\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: Could not load this library: D:\software\miniconda\envs\env_test_1\Lib\site-packages\torch_cluster\_version_cuda.pyd
  import torch_geometric.typing
d:\software\miniconda\envs\env_test_1\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: Could not load this library: D:\software\miniconda\envs\env_test_1\Lib\site-packages\torch_spline_conv\_version_cuda.pyd
  import torch_geometric.

Loading checkpoint from: Checkpoints\model_epoch_15.pt
HTGPConfig(num_atom_types=55, hidden_dim=128, num_layers=2, cutoff=7.0, num_rbf=10, atom_types_map=[1, 5, 6, 7, 8, 9, 15, 16, 17, 35, 53], use_L0=True, use_L1=True, use_L2=True, use_gating=True, avg_neighborhood=141.53002603583755, use_long_range=False, active_paths={(0, 0, 0, 'prod'): True, (0, 1, 1, 'prod'): True, (0, 2, 2, 'prod'): True, (1, 0, 1, 'prod'): True, (1, 1, 0, 'dot'): True, (1, 1, 1, 'cross'): True, (1, 1, 2, 'outer'): True, (2, 0, 2, 'prod'): True, (2, 1, 1, 'mat_vec'): True, (1, 2, 1, 'vec_mat'): True, (2, 2, 0, 'double_dot'): True, (2, 2, 2, 'mat_mul_sym'): True})
✅ Found configuration in checkpoint! Using saved config.


In [2]:
model = HTGPModel(config)

if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
    state_dict = checkpoint['model_state_dict']
else:
    # 兼容旧版本直接保存 model.state_dict() 的情况
    state_dict = checkpoint
    # 去除 DDP 的 "module." 前缀
    
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('module.'):
        new_state_dict[k[7:]] = v 
    else:
        new_state_dict[k] = v

# 加载权重
try:
    model.load_state_dict(new_state_dict, strict=True)
    print("✅ Weights loaded successfully!")
except RuntimeError as e:
    print(f"❌ Weight loading failed: {e}")
    # 打印出具体的 shape 不匹配信息，方便调试
    raise e 

✅ Weights loaded successfully!


In [3]:
calc_cutoff = getattr(config, 'cutoff', 7.0)
calc = HTGP_Calculator(model, cutoff=calc_cutoff, device=device)

In [6]:
from ase.io import read
demo_path = r"ABABAK.xyz"
atoms = read(demo_path)
ref_force = atoms.arrays["REF_forces"]

atoms.set_calculator(calc)
energy = atoms.get_potential_energy()
print(energy)
force = atoms.get_forces()
print(force)
stress = atoms.get_stress()
print(stress)

-807.5679321289062
[[ 4.09519896e-02 -4.55977142e-01  3.53266299e-03]
 [-4.40737829e-02  4.54048008e-01 -1.37493014e-04]
 [-7.77128339e-03 -1.32099077e-01 -9.73106623e-02]
 [ 6.66292757e-03  1.30730748e-01  9.53859985e-02]
 [ 2.36422606e-02  7.58826733e-04  1.01725757e-02]
 [-2.21522525e-02 -1.40920281e-04 -9.93072987e-03]
 [ 8.04017857e-02 -4.29346025e-01  9.27698165e-02]
 [-7.89940208e-02  4.31401253e-01 -9.36524570e-02]
 [-3.22314888e-01  2.58301675e-01 -2.37985328e-01]
 [ 3.23524624e-01 -2.55477607e-01  2.38495559e-01]
 [-9.32195336e-02  1.80938885e-01 -4.32695895e-02]
 [ 9.47120190e-02 -1.81402341e-01  4.25656438e-02]
 [ 6.05017990e-02 -1.98675334e-01  2.13229340e-02]
 [-5.94024137e-02  1.98504850e-01 -2.17716210e-02]
 [ 1.30964100e-01  9.99880731e-02  9.20976996e-02]
 [-1.31828278e-01 -1.00571081e-01 -9.26923901e-02]
 [ 1.40758380e-01  3.13051879e-01  1.49978697e-01]
 [-1.40161872e-01 -3.13241810e-01 -1.49967194e-01]
 [ 1.32755302e-02 -2.99080089e-02  2.18814671e-01]
 [-1.1001944

C:\Users\1\AppData\Local\Temp\ipykernel_2656\3240654771.py:6: FutureWarning: Please use atoms.calc = calc
  atoms.set_calculator(calc)


In [ ]:
# structure optimization demo
from ase.io import read, write
from ase.optimize import BFGS, LBFGS, FIRE, GPMin, MDMin, QuasiNewton
from ase.filters import FrechetCellFilter

raw_struct =  r"CSP_virtual_structure.cif"
crystal =  read(raw_struct)
crystal.calc = calc
sf = FrechetCellFilter(crystal, scalar_pressure=0.0006)
optimizer = QuasiNewton(sf)
optimizer.run(fmax=0.01, steps=3000)
crystal.write(r"CSP_virtual_structure_press.cif")

crystal = read(r"CSP_virtual_structure_press.cif")
crystal.calc = calc
sf = FrechetCellFilter(crystal)
optimizer = BFGS(sf)
optimizer.run(fmax=0.01, steps=3000)
crystal.write(r"CSP_virtual_structure_final.cif")

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 21:29:25     -341.286035       6.2985


d:\software\miniconda\envs\env_test_1\Lib\site-packages\scipy\_lib\_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 4.726475621142667e-13
  return f(*arrays, *other_args, **kwargs)


BFGSLineSearch:    1[  2] 21:29:25     -341.613726       3.4478


d:\software\miniconda\envs\env_test_1\Lib\site-packages\scipy\_lib\_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 4.733132060635183e-13
  return f(*arrays, *other_args, **kwargs)


BFGSLineSearch:    2[  4] 21:29:26     -341.819629       1.1014


...

d:\software\miniconda\envs\env_test_1\Lib\site-packages\scipy\_lib\_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 4.726441070033693e-13
  return f(*arrays, *other_args, **kwargs)
d:\software\miniconda\envs\env_test_1\Lib\site-packages\scipy\_lib\_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 4.75514369412803e-13
  return f(*arrays, *other_args, **kwargs)


BFGS:   26 21:32:07     -346.932465        0.014372
BFGS:   27 21:32:07     -346.932495        0.012595


d:\software\miniconda\envs\env_test_1\Lib\site-packages\scipy\_lib\_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 4.715376431070281e-13
  return f(*arrays, *other_args, **kwargs)
d:\software\miniconda\envs\env_test_1\Lib\site-packages\scipy\_lib\_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 4.668822164670942e-13
  return f(*arrays, *other_args, **kwargs)


BFGS:   28 21:32:07     -346.932526        0.008862


d:\software\miniconda\envs\env_test_1\Lib\site-packages\scipy\_lib\_util.py:1233: RuntimeWarning: logm result may be inaccurate, approximate err = 4.746100702343875e-13
  return f(*arrays, *other_args, **kwargs)
